In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

In [2]:
# Khởi tạo trình duyệt Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# Đọc file CSV đầu vào
input_csv = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/crawldata3seealllinks.csv"  
df_input = pd.read_csv(input_csv)

In [4]:
# Danh sách để lưu dữ liệu crawl được
data = []

In [5]:
# Duyệt qua từng link trong file CSV
for index, row in df_input.iterrows():
    resume_link = row['resume_link']
    print(f"Đang crawl link: {resume_link}")
    
    try:
        # Truy cập link
        driver.get(resume_link)
        time.sleep(2)  # Chờ trang tải hoàn toàn
        
        # 1. Crawl từ section id="top-examples" class="cat-res-grid"
        try:
            top_examples = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "top-examples"))
            )
            blade_content = top_examples.find_element(By.CLASS_NAME, "blade-content")
            cols = blade_content.find_elements(By.CLASS_NAME, "col")
            
            for col in cols:
                try:
                    h3 = col.find_element(By.TAG_NAME, "h3")
                    a_tag = h3.find_element(By.TAG_NAME, "a")
                    href = a_tag.get_attribute("href")
                    content = a_tag.text.strip()
                    data.append({"resume_link": href, "Category": content})
                    print(f"Đã crawl từ top-examples: {href} - {content}")
                except:
                    continue
        except:
            print(f"Không tìm thấy section 'top-examples' trong {resume_link}, bỏ qua.")
        
        # 2. Crawl từ section id="more-examples" class="jbt-link-list"
        try:
            more_examples = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "more-examples"))
            )
            jbt_container = more_examples.find_element(By.CLASS_NAME, "jbt-container")
            ul = jbt_container.find_element(By.CLASS_NAME, "container-sm")
            li_tags = ul.find_elements(By.TAG_NAME, "li")
            
            for li in li_tags:
                try:
                    a_tag = li.find_element(By.TAG_NAME, "a")
                    href = a_tag.get_attribute("href")
                    content = a_tag.text.strip()
                    data.append({"resume_link": href, "Category": content})
                    print(f"Đã crawl từ more-examples: {href} - {content}")
                except:
                    continue
        except:
            print(f"Không tìm thấy section 'more-examples' trong {resume_link}, bỏ qua.")
        
        # 3. Crawl từ section id="resume-examples" class="jbt-link-list brown"
        try:
            resume_examples = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "resume-examples"))
            )
            jbt_container = resume_examples.find_element(By.CLASS_NAME, "jbt-container")
            ul = jbt_container.find_element(By.CLASS_NAME, "container-sm")
            li_tags = ul.find_elements(By.TAG_NAME, "li")
            
            for li in li_tags:
                try:
                    a_tag = li.find_element(By.TAG_NAME, "a")
                    href = a_tag.get_attribute("href")
                    content = a_tag.text.strip()
                    data.append({"resume_link": href, "Category": content})
                    print(f"Đã crawl từ resume-examples: {href} - {content}")
                except:
                    continue
        except:
            print(f"Không tìm thấy section 'resume-examples' trong {resume_link}, bỏ qua.")
        
    except Exception as e:
        print(f"Lỗi khi crawl link {resume_link}: {e}")
        continue

Đang crawl link: https://www.myperfectresume.com/cv/examples/accounting
Không tìm thấy section 'top-examples' trong https://www.myperfectresume.com/cv/examples/accounting, bỏ qua.
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/trainee - Accountant Trainee CV
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/officer - Accounts Officer CV
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/audit-assistant - Audit Assistant CV
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/certified-public-accountant - Certified Public Accountant CV
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/cost-accountant - Cost Accountant CV
Đã crawl từ more-examples: https://www.myperfectresume.com/cv/examples/accounting/project-accountant - Project Accountant CV
Đã crawl từ resume-examples: https://www.myperfectresume.com/resume/examples/accounting/

In [6]:
# Đóng trình duyệt
driver.quit()

In [7]:
# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(data)

In [9]:
df.head(5)

,resume_link,Category
0,https://www.myperfectresume.com/cv/examples/ac...,Accountant Trainee CV
1,https://www.myperfectresume.com/cv/examples/ac...,Accounts Officer CV
2,https://www.myperfectresume.com/cv/examples/ac...,Audit Assistant CV
3,https://www.myperfectresume.com/cv/examples/ac...,Certified Public Accountant CV
4,https://www.myperfectresume.com/cv/examples/ac...,Cost Accountant CV


In [10]:
# Lưu vào file CSV
output_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/indata3seealllinks.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Dữ liệu đã được lưu vào file: {output_file}")
print(f"Tổng số link crawl được: {len(df)}")

Dữ liệu đã được lưu vào file: D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/indata3seealllinks.csv
Tổng số link crawl được: 808
